In [ ]:
from typing import Dict, Union

import numpy as np
import pandas as pd
import os
import ast

from itertools import zip_longest

from datetime import datetime
from pytz import timezone

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
if not os.path.exists('/content/drive/MyDrive/'):
    raise Exception("ERROR: Mount Google Drive before continuing!")

BASE_DIR = '/content/drive/MyDrive/Data Science for Social Good - Spring 2022/data/'
SCRAPE_DIR = BASE_DIR + 'scraped_files/'
DATA_DIR = SCRAPE_DIR + 'DATA/'
CLEAN_DIR = SCRAPE_DIR + 'CLEAN/'
MERGED_DIR = SCRAPE_DIR + 'MERGED_COUNTIES/'

CURRENT_DATE = datetime.now(timezone('US/Eastern')).strftime('%m-%d-%Y')

# Formatter for Consistency

---

We aren't sure if there can be keys other than the ones listed below. 

For previously unseen keys, we use str.upper() to give them a default value because we want to be able to automate the web scraper, so we want to handle errors without stopping.

Any previously unseen keys should be added to the dictionary along with a value to ensure consistency.

---

## **Ensure all tuples end with a comma**

If we have a tuple of a single element, it should look like:  
(MY_ELEMENT,)

If we do (MY_ELEMENT) this will get parsed character by character, since (MY_ELEMENT) is equivalent to the string MY_ELEMENT.

In [ ]:
CASES_CHARGES_SHARED_KEYS = ['caseNo', 'bondAmount', 'courtTime', 'bondType']
INCONSISTENT_COLUMNS = ['Race', 'Sex', 'Eye Color', 'Hair Color', 'Complexion', 'Facial Hair']
UNKNOWN_VAL = 'N/A'

# Some columns are inconsistently named despite representing the same type of data
MISNAMED_COLUMN_MAPPINGS = {
    'Intake_num': 'Booking Number',
    'Arrest Agency': 'Arresting Agency',
    'Hair': 'Hair Color',
    'Eye': 'Eye Color',
}

# We use many-to-one mappings (Dict[Tuple, str]) for convenience - easier to add mappings
# We convert them to one-to-one dictionaries later
RACE_MAPPINGS = {
    ('B', 'Black', 'BLACK', 'African American',): 'BLACK',
    ('W', 'White', 'WHITE', 'Caucasian',): 'WHITE',
    ('H', 'Hispanic', 'HISPANIC',): 'HISPANIC',
    ('A', 'ASIAN', 'Indian', 'INDIAN',): 'ASIAN',
    ('AMERICAN', 'Native American',): 'INDIGENOUS',  # Unsure about 'AMERICAN'
    ('MIDDLE EASTERN',): 'MIDDLE EASTERN',
    ('Other', 'OTHER',): 'OTHER',
    ('U', 'UNKNOWN', 'UNDECIDED', 'N/A',): UNKNOWN_VAL,
}

GENDER_MAPPINGS = {
    ('F', 'Female', 'FEMALE',): 'FEMALE',
    ('M', 'Male', 'MALE',): 'MALE',
    ('', 'N/A'): UNKNOWN_VAL,
}

EYE_COLOR_MAPPINGS = {
    ('GRN', 'Green', 'GREEN',): 'GREEN',
    ('HAZ', 'Hazel', 'HAZEL',): 'HAZEL',
    ('BLU', 'Blue', 'BLUE',): 'BLUE',
    ('BRO', 'Brown', 'BROWN',): 'BROWN',
    ('DARK BROWN',): 'DARK BROWN',
    ('GRY', 'Gray', 'GRAY',): 'GREY',
    ('BLK', 'Black', 'BLACK',): 'BLACK',
    ('MAROON',): 'MAROON',
    ('Other', 'OTHER',): 'OTHER',
    ('UNKN', 'Unknown', 'UNKNOWN',): UNKNOWN_VAL,
}

HAIR_COLOR_MAPPINGS = {
    ('BLK', 'Black', 'BLACK',): 'BLACK',
    ('BRO', 'Brown', 'BROWN',): 'BROWN',
    ('Blue', 'BLUE',): 'BLUE',
    ('GRY', 'Gray', 'GRAY', 'GREY', 'GRAY/GRA',): 'GREY',
    ('WHI', 'White', 'WHITE',): 'WHITE',
    ('Red', 'RED', 'RED/AUBU', 'RED OR A', 'Auburn', 'AUBURN',): 'RED',
    ('BAL', 'Bald', 'BALD', 'BALD/BAL',): 'BALD',
    ('SDY', 'Sandy', 'SANDY',): 'SANDY',
    ('BLN', 'Blond', 'BLOND', 'Blonde', 'BLONDE',): 'BLONDE',
    ('ORANGE',): 'ORANGE',
    ('GREEN',): 'GREEN',
    ('Pink', 'PINK',): 'PINK',
    ('Purple', 'PLE', 'PURPLE',): 'PURPLE',
    ('MULTICOLORED', 'MIXED',): 'MULTICOLORED',
    ('FROSTED',): 'FROSTED',
    ('Other', 'OTHER',): 'OTHER',
    ('XXX', 'Unknown', 'UNKNOWN', 'NONE',): UNKNOWN_VAL,
}

COMPLEXION_MAPPINGS = {
    ('ALBINO',): 'ALBINO',
    ('FAIR',): 'FAIR',
    ('LIGHT',): 'LIGHT',
    ('DARK',): 'DARK',
    ('BLACK',): 'BLACK',
    ('MEDIUM',): 'MEDIUM',
    ('OLIVE',): 'OLIVE',
    ('RUDDY',): 'RUDDY',
    ('DARK BRO',): 'DARK BROWN',
    ('MEDIUM B',): 'MEDIUM BLACK',
    ('LIGHT BR',): 'LIGHT BROWN',
    ('YELLOW',): 'YELLOW',
    ('UNKNOWN',): UNKNOWN_VAL,
}

FACIAL_HAIR_MAPPINGS = {
    ('BEARD',): 'BEARD',
    ('CLEAN SH', 'NONE'): 'CLEAN SHAVED',
    ('FU MANCH',): 'FU MANCHU',
    ('FULL BEA',): 'FULL BEARD',
    ('FUZZ',): 'FUZZ',
    ('GOATEE',): 'GOATEE',
    ('LOWER LI',): 'LOWER LIP',
    ('MUSTACHE',): 'MUSTACHE',
    ('SIDEBURN',): 'SIDEBURN',
    ('THIN BEA',): 'THIN BEARD',
    ('UNSHAVEN', 'UNKNOWN', 'NOT APPL',): UNKNOWN_VAL,
}

In [ ]:
class Formatter():
    class FormatterDict(dict):
        """Custom class to handle missing keys by subclassing dict. Currently returns str.upper() for missing keys."""
        def __missing__(self, key: str) -> str:
            print("WARNING: Missing key:", key)
            return str.upper(key)

    def __init__(self) -> None:
        self.race_mappings = self._convert_mapping_to_dict(RACE_MAPPINGS)
        self.gender_mappings = self._convert_mapping_to_dict(GENDER_MAPPINGS)
        self.eye_color_mappings = self._convert_mapping_to_dict(EYE_COLOR_MAPPINGS)
        self.hair_color_mappings = self._convert_mapping_to_dict(HAIR_COLOR_MAPPINGS)
        self.complexion_mappings = self._convert_mapping_to_dict(COMPLEXION_MAPPINGS)
        self.facial_hair_mappings = self._convert_mapping_to_dict(FACIAL_HAIR_MAPPINGS)

    def get_mappings(self, name: str) -> Dict:
        if name == 'Race':
            return self.race_mappings
        elif name == 'Sex':
            return self.gender_mappings
        elif name == 'Eye Color':
            return self.eye_color_mappings
        elif name == 'Hair Color':
            return self.hair_color_mappings
        elif name == 'Complexion':
            return self.complexion_mappings
        elif name == 'Facial Hair':
            return self.facial_hair_mappings
        else:
            raise Exception("ERROR: Unrecognized name", name)

    def format_column(self, column: pd.Series) -> pd.Series:
        # Ignore columns that are completely null
        if column.isnull().all() == True:
            return column

        # Remove any trailing whitespace
        column = column.str.rstrip()

        mappings = self.get_mappings(column.name)
        return column.map(mappings, na_action='ignore')

    def format_row(self, row: pd.Series) -> pd.Series:

        def format_str(value: str) -> str:
            if value == '':
                return None
            return value

        def format_money(value: Union[int, str, float]) -> str:
            # print(type(value)) 
            # Somehow gets int/float sometimes, function takes string but type(value) returns float/int
            if type(value) not in [str, int, float]:
                print("Money value has different data type ", type(value))
                return value
            if value == 'N/A':
                value = '0.00'

            # We sometimes have commas in the bondAmount (ie. Jackson) so we get rid of those and recombine
            if "," in str(value):
                temp = value.split(",")
                value = "".join(temp)
            return f'{float(value):g}'

        def format_serialized_dict(column_name: str) -> pd.Series:
            row[column_name] = ast.literal_eval(row[column_name])
            row[column_name] = [{k: format_str(v) for k, v in x.items()} for x in row[column_name]]
            for x in row[column_name]:
                x['bondAmount'] = format_money(x['bondAmount'])
            return row

        # Both Cases and Charges do not appear as columns
        if 'Cases' not in row.index and 'Charges' not in row.index:
            return row

        # If both columns are there (misses one of each)
        elif set(['Cases', 'Charges']).issubset(row.index):

            # Read serialized list of dicts
            # We can't use the json package because the JSON isn't serialized properly
            # We have: {'name': 'value', 'name_2': None}
            # Should be: {"name": "value", "name_2": null}
            row['Cases'] = ast.literal_eval(row['Cases'])
            row['Charges'] = ast.literal_eval(row['Charges'])

            if len(row['Cases']) == 0:
                row.drop('Cases', inplace=True)
                return row
            if len(row['Cases']) != len(row['Charges']):
                # print("Different length", len(row['Cases']), len(row['Charges']), row['Cases'], row['Charges'])
                
                # Duplicate the entry in row['Cases'] to match row['Charges']
                if len(row['Cases']) == 1:
                    row['Cases'] *= len(row['Charges'])
                else:
                    # print("Not 1:X ratio", len(row['Cases']), len(row['Charges']), row['Cases'], row['Charges'])
                    
                    row['Cases'] = [{k: format_str(v) for k, v in x.items()} for x in row['Cases']]
                    row['Charges'] = [{k: format_str(v) for k, v in x.items()} for x in row['Charges']]

                    combined = []
                    for case, charge in zip_longest(row['Cases'], row['Charges']):

                        # The caseNo is the same
                        if case and charge and case['caseNo'] == charge['caseNo'] and charge['caseNo'] != '':

                            # Remove shared keys if the values between the two columns do not match and to merge the remaining key value pairs
                            shared_keys = case.keys() & charge.keys()
                            if len(shared_keys) != 0:
                                for key in shared_keys:
                                    if case[key] != charge[key]:
                                        case.pop(key)
                                        charge.pop(key)
                                    else:
                                        # Keep one
                                        case.pop(key)
                            
                            # Merge by adding the different key value pairs
                            case.update(charge)
                            combined.append(case)

                        else:
                            if case:
                                combined.append(case)
                            if charge:
                                combined.append(charge)

                    row['Charges'] = combined
                    row.drop('Cases', inplace=True)
                    # print(len(row['Charges']))
                    return row
            
            # Equal cases and charges length case, everything is 1:1 and in order
            # Update values for consistency
            row['Cases'] = [{k: format_str(v) for k, v in x.items()} for x in row['Cases']]
            row['Charges'] = [{k: format_str(v) for k, v in x.items()} for x in row['Charges']]

            combined = []
            for case, charge in zip(row['Cases'], row['Charges']):
                case['bondAmount'] = format_money(case['bondAmount'])
                charge['bondAmount'] = format_money(charge['bondAmount'])

                # Remove shared keys
                for key in CASES_CHARGES_SHARED_KEYS:
                    if case[key] != charge[key]:
                        #print("WARNING: Different", key, case[key], charge[key])
                        #print(case)
                        #print(charge)

                        case.pop(key)
                        charge.pop(key)
                    else:
                        case.pop(key)

                # Check if case and charge share any keys
                shared_keys = case.keys() & charge.keys()
                if len(shared_keys) != 0:
                    print("WARNING: Duplicate keys", shared_keys)
                    for key in shared_keys:
                        if case[key] != charge[key]:
                            case.pop(key)
                            charge.pop(key)
                        else:
                            case.pop(key)
                case.update(charge)
                combined.append(case)

            row['Charges'] = combined
            row.drop('Cases', inplace=True)

            return row

        # Only Cases exists as a column
        elif 'Cases' in row.index and 'Charges' not in row.index:
            if len(row['Cases']) == 0:
                return row
            
            return format_serialized_dict('Cases')

        # Only Charges exists as a column
        elif 'Cases' not in row.index and 'Charges' in row.index:
            if len(row['Charges']) == 0:
                return row

            return format_serialized_dict('Charges')

    def format_df(self, df: pd.DataFrame) -> pd.DataFrame:
        df.rename(MISNAMED_COLUMN_MAPPINGS, axis='columns', inplace=True)

        # Merge Cases and Charges
        df = df.apply(self.format_row, axis=1)

        # Remove any columns not found in the dataframe
        columns_to_fix = [col for col in INCONSISTENT_COLUMNS if col in df.columns]

        # Fix inconsistent data formatting
        df[columns_to_fix] = df[columns_to_fix].apply(self.format_column)
        return df

    def _convert_mapping_to_dict(self, many_to_one: Dict) -> Dict:
        """Convert a many-to-one dictionary to a one-to-one dictionary.
        For example:
        {('key_1', 'key_2'): 'val'} -> {'key_1': 'val', 'key_2': 'val'}
        """
        one_to_one = self.FormatterDict()
        for key_tuple, val in many_to_one.items():
            # Safety check for user error
            if type(key_tuple) != tuple:
                raise Exception("ERROR: Key tuple entered incorrectly!", key_tuple)

            for key in key_tuple:
                one_to_one[key] = val
        return one_to_one

# Helper Functions

---



In [ ]:
def clear_directory(dir_name: str) -> None:
    for sub_dir, dirs, files in os.walk(dir_name):
        for filename in files:
            os.remove(f'{sub_dir}/{filename}')

# Clean Data

---

This will automatically clean the data for the current day. If needed, this can be expanded to clean data from previous days.

In [ ]:
# DATES_TO_CLEAN = os.listdir(DATA_DIR)
DATES_TO_CLEAN = [CURRENT_DATE]
formatter = Formatter()

for date_to_clean in DATES_TO_CLEAN:
    print("Cleaning date:", date_to_clean)
    dir_to_clean = f'{DATA_DIR}{date_to_clean}'
    for sub_dir, _, files in os.walk(dir_to_clean):
        # Create directory if needed and remove existing one
        new_dir = f'{CLEAN_DIR}{date_to_clean}'
        os.makedirs(new_dir, exist_ok=True)
        clear_directory(new_dir)

        for filename in files:
            cur_file = f'{sub_dir}/{filename}'
            df = pd.read_csv(cur_file)

            # Ignore the index column present in some files
            # https://stackoverflow.com/a/43983654
            df.drop(df.columns[df.columns.str.contains('^Unnamed')], axis=1, inplace=True)

            df = formatter.format_df(df)
            df.to_csv(f'{new_dir}/{filename}', index=False)

print("Finished cleaning")

# Merge Data

---

This will automatically merge the data for the current day. If needed, this can be expanded to merge data from previous days.

In [ ]:
# DATES_TO_MERGE = os.listdir(CLEAN_DIR)
DATES_TO_MERGE = [CURRENT_DATE]

for date_to_merge in DATES_TO_MERGE:
    print("Merging:", date_to_merge)
    dir_to_merge = f'{CLEAN_DIR}{date_to_merge}'
    for sub_dir, _, files in os.walk(dir_to_merge):
        # Create directory if needed and remove existing one
        new_dir = f'{MERGED_DIR}{date_to_merge}'
        os.makedirs(new_dir, exist_ok=True)
        clear_directory(new_dir)

        df_list = []
        for filename in files:
            cur_file = f'{sub_dir}/{filename}'
            df = pd.read_csv(cur_file)
            
            # Add the county to the dataframe
            df['County'] = filename.split('_')[-1].split('.')[0]
            df_list.append(df)

        df_merged = pd.concat(df_list, axis=0)
        df_merged.to_csv(f'{new_dir}/{date_to_merge}_Merged.csv', index=False)
        print("Merged", len(df_list), "counties")

print("Finished merging")